## Setup

In [ ]:
!pip install -U oci

## Generative AI - Inference(via LangChain)

In [ ]:
from os import getenv
from dotenv import load_dotenv

from langchain_community.llms.oci_generative_ai import OCIGenAI

load_dotenv()

COMPARTMENT_ID = getenv("COMPARTMENT_ID")
SERVICE_ENDPOINT = getenv("GEN_AI_INFERENCE_ENDPOINT")

### Using API Key

In [ ]:
llm_cohere_command = OCIGenAI(
    auth_profile="LONDON",
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command"
)

response = llm_cohere_command.invoke("Tell me a joke", temperature=0.7)

response

In [ ]:
llm_cohere_command_light = OCIGenAI(
    auth_profile="LONDON",
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command-light"
)

response = llm_cohere_command_light.invoke("Tell me a joke", temperature=0.7)

response

In [ ]:
llm_meta = OCIGenAI(
    auth_profile="LONDON",
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="meta.llama-2-70b-chat"
)

llm_meta.invoke("Tell me a joke")

### Using Instance Principal

In [ ]:
llm_cohere = OCIGenAI(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command",
    model_kwargs={
        "temperature": 0.7
    }
)

llm_cohere.invoke("Tell me a joke")

## Prompt Template

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

llm_cohere = OCIGenAI(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command"
)

prompt = PromptTemplate(input_variables=["query"], template="{query}")
llm_chain = LLMChain(llm=llm_cohere, prompt=prompt)

response = llm_chain.invoke("what is the capital of france?")

response

## Simple RAG

In [ ]:
!pip install faiss-cpu

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS

In [ ]:
embeddings = OCIGenAIEmbeddings(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=SERVICE_ENDPOINT,
    model_id="cohere.embed-english-v3.0",
    compartment_id=COMPARTMENT_ID
)

vectorstore = FAISS.from_texts(
    [
        "Larry Ellison co-founded Oracle Corporation in 1977 with Bob Miner and Ed Oates.",
        "Oracle Corporation is an American multinational computer technology company headquartered in Austin, Texas, United States.",
    ],
    embedding=embeddings
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template=template)
llm_cohere = OCIGenAI(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=SERVICE_ENDPOINT,
    model_id="cohere.command",
    compartment_id=COMPARTMENT_ID
)

chain = (
    { "context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm_cohere
    | StrOutputParser()
)

print(chain.invoke("when was oracle founded?"))
print(chain.invoke("where is oracle headquartered?"))